In [ ]:
!gcc --version

gcc (Ubuntu 11.4.0-1ubuntu1~22.04.2) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



**Part 1: Matrix Multiplication on the CPU**

In [ ]:
%%writefile matrix_cpu.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

static void matrixMultiplyCPU(const float *A, const float *B, float *C, int N) {
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            float sum = 0.0f;
            for (int k = 0; k < N; k++) {
                sum += A[i * N + k] * B[k * N + j];
            }
            C[i * N + j] = sum;
        }
    }
}

int main(int argc, char **argv) {
    int N = (argc > 1) ? atoi(argv[1]) : 1024;
    size_t size = (size_t)N * (size_t)N * sizeof(float);

    float *A = (float *)malloc(size);
    float *B = (float *)malloc(size);
    float *C = (float *)malloc(size);

    if (!A || !B || !C) {
        fprintf(stderr, "malloc failed for N=%d (need %.2f MB per matrix)\n",
                N, (double)size / (1024.0 * 1024.0));
        free(A); free(B); free(C);
        return 1;
    }

    // init with pseudo-random floats in [0, 0.99]
    srand(0); // fixed seed for reproducibility
    for (int i = 0; i < N * N; i++) {
        A[i] = (rand() % 100) / 100.0f;
        B[i] = (rand() % 100) / 100.0f;
    }

    clock_t start = clock();
    matrixMultiplyCPU(A, B, C, N);
    clock_t end = clock();

    double elapsed = (double)(end - start) / CLOCKS_PER_SEC;
    printf("CPU execution time (N=%d): %.6f seconds\n", N, elapsed);

    free(A); free(B); free(C);
    return 0;
}

Writing matrix_cpu.c


In [ ]:
!gcc matrix_cpu.c -o matrix_cpu -O2

In [ ]:
!./matrix_cpu 512
!./matrix_cpu 1024
!./matrix_cpu 2048

CPU execution time (N=512): 0.297522 seconds
CPU execution time (N=1024): 3.436814 seconds
CPU execution time (N=2048): 76.421317 seconds


**Part 2: Introduction to CUDA Programming**

In [ ]:
!nvidia-smi

Sat Jan 31 20:53:20 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             46W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
%%writefile matrix_gpu.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

__global__ void matrixMultiplyGPU(const float *A, const float *B, float *C, int N) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < N && col < N) {
        float sum = 0.0f;
        for (int k = 0; k < N; k++) {
            sum += A[row * N + k] * B[k * N + col];
        }
        C[row * N + col] = sum;
    }
}

int main(int argc, char **argv) {
    int N = (argc > 1) ? atoi(argv[1]) : 1024;
    size_t size = (size_t)N * (size_t)N * sizeof(float);

    float *hA = (float*)malloc(size);
    float *hB = (float*)malloc(size);
    float *hC = (float*)malloc(size);

    srand(0);
    for (int i = 0; i < N * N; i++) {
        hA[i] = (rand() % 100) / 100.0f;
        hB[i] = (rand() % 100) / 100.0f;
    }

    float *dA, *dB, *dC;
    cudaMalloc((void**)&dA, size);
    cudaMalloc((void**)&dB, size);
    cudaMalloc((void**)&dC, size);

    cudaMemcpy(dA, hA, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dB, hB, size, cudaMemcpyHostToDevice);

    dim3 block(16, 16);
    dim3 grid((N + 15) / 16, (N + 15) / 16);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);
    matrixMultiplyGPU<<<grid, block>>>(dA, dB, dC, N);
    cudaEventRecord(stop);

    cudaDeviceSynchronize();

    float ms = 0.0f;
    cudaEventElapsedTime(&ms, start, stop);

    cudaMemcpy(hC, dC, size, cudaMemcpyDeviceToHost);

    printf("GPU kernel time (N=%d): %.3f ms (%.6f seconds)\n", N, ms, ms / 1000.0);

    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    cudaFree(dA); cudaFree(dB); cudaFree(dC);
    free(hA); free(hB); free(hC);
    return 0;
}

Writing matrix_gpu.cu


In [ ]:
!nvcc matrix_gpu.cu -o matrix_gpu -O2 -arch=sm_80

In [ ]:
!./matrix_gpu 512
!./matrix_gpu 1024
!./matrix_gpu 2048

GPU kernel time (N=512): 0.194 ms (0.000194 seconds)
GPU kernel time (N=1024): 0.939 ms (0.000939 seconds)
GPU kernel time (N=2048): 7.016 ms (0.007016 seconds)


**Part 3: Running CUDA on Google Cloud**

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


**Part 4: Optimizing CUDA Code**

In [ ]:
%%writefile matrix_tiled.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

#define TILE_WIDTH 16

__global__ void matrixMultiplyTiled(const float *A, const float *B, float *C, int N) {
    __shared__ float ds_A[TILE_WIDTH][TILE_WIDTH];
    __shared__ float ds_B[TILE_WIDTH][TILE_WIDTH];

    int tx = threadIdx.x, ty = threadIdx.y;
    int Row = blockIdx.y * TILE_WIDTH + ty;
    int Col = blockIdx.x * TILE_WIDTH + tx;

    float Pvalue = 0.0f;
    int numTiles = (N + TILE_WIDTH - 1) / TILE_WIDTH;

    for (int m = 0; m < numTiles; m++) {
        int A_col = m * TILE_WIDTH + tx;
        int B_row = m * TILE_WIDTH + ty;

        ds_A[ty][tx] = (Row < N && A_col < N) ? A[Row * N + A_col] : 0.0f;
        ds_B[ty][tx] = (B_row < N && Col < N) ? B[B_row * N + Col] : 0.0f;

        __syncthreads();

        for (int k = 0; k < TILE_WIDTH; k++) {
            Pvalue += ds_A[ty][k] * ds_B[k][tx];
        }

        __syncthreads();
    }

    if (Row < N && Col < N) {
        C[Row * N + Col] = Pvalue;
    }
}

int main(int argc, char **argv) {
    int N = (argc > 1) ? atoi(argv[1]) : 1024;
    size_t size = (size_t)N * (size_t)N * sizeof(float);

    float *hA = (float*)malloc(size);
    float *hB = (float*)malloc(size);
    float *hC = (float*)malloc(size);

    srand(0);
    for (int i = 0; i < N * N; i++) {
        hA[i] = (rand() % 100) / 100.0f;
        hB[i] = (rand() % 100) / 100.0f;
    }

    float *dA, *dB, *dC;
    cudaMalloc((void**)&dA, size);
    cudaMalloc((void**)&dB, size);
    cudaMalloc((void**)&dC, size);

    cudaMemcpy(dA, hA, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dB, hB, size, cudaMemcpyHostToDevice);

    dim3 block(TILE_WIDTH, TILE_WIDTH);
    dim3 grid((N + TILE_WIDTH - 1) / TILE_WIDTH, (N + TILE_WIDTH - 1) / TILE_WIDTH);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);
    matrixMultiplyTiled<<<grid, block>>>(dA, dB, dC, N);
    cudaEventRecord(stop);

    cudaDeviceSynchronize();

    float ms = 0.0f;
    cudaEventElapsedTime(&ms, start, stop);

    cudaMemcpy(hC, dC, size, cudaMemcpyDeviceToHost);

    printf("Tiled GPU kernel time (N=%d, TILE=%d): %.3f ms (%.6f seconds)\n",
           N, TILE_WIDTH, ms, ms / 1000.0);

    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    cudaFree(dA); cudaFree(dB); cudaFree(dC);
    free(hA); free(hB); free(hC);
    return 0;
}

Writing matrix_tiled.cu


In [ ]:
!nvcc matrix_tiled.cu -o matrix_tiled -O2 -arch=sm_80

In [ ]:
!./matrix_tiled 512
!./matrix_tiled 1024
!./matrix_tiled 2048

Tiled GPU kernel time (N=512, TILE=16): 0.144 ms (0.000144 seconds)
Tiled GPU kernel time (N=1024, TILE=16): 0.649 ms (0.000649 seconds)
Tiled GPU kernel time (N=2048, TILE=16): 4.732 ms (0.004732 seconds)


**Part 5: Performance Comparison**

In [ ]:
import pandas as pd

cpu_sec = {512: 0.297522, 1024: 3.436814, 2048: 76.421317}
naive_ms = {512: 0.194, 1024: 0.939, 2048: 7.016}
tiled_ms = {512: 0.144, 1024: 0.649, 2048: 4.732}

runtime_table = pd.DataFrame({
    "Implementation": ["CPU (C)", "Naïve CUDA", "Optimized CUDA (Tiled)"],
    "N=512":  [f"{cpu_sec[512]:.6f} sec", f"{naive_ms[512]:.3f} ms", f"{tiled_ms[512]:.3f} ms"],
    "N=1024": [f"{cpu_sec[1024]:.6f} sec", f"{naive_ms[1024]:.3f} ms", f"{tiled_ms[1024]:.3f} ms"],
    "N=2048": [f"{cpu_sec[2048]:.6f} sec", f"{naive_ms[2048]:.3f} ms", f"{tiled_ms[2048]:.3f} ms"],
})

speedup_table = pd.DataFrame({
    "N": [512, 1024, 2048],
    "Speedup (CPU / Naïve CUDA)": [
        cpu_sec[512]  / (naive_ms[512]  / 1000),
        cpu_sec[1024] / (naive_ms[1024] / 1000),
        cpu_sec[2048] / (naive_ms[2048] / 1000),
    ],
    "Speedup (CPU / Optimized CUDA)": [
        cpu_sec[512]  / (tiled_ms[512]  / 1000),
        cpu_sec[1024] / (tiled_ms[1024] / 1000),
        cpu_sec[2048] / (tiled_ms[2048] / 1000),
    ],
}).round(2)

print("=== Part 5: Runtime Comparison Table ===")
display(runtime_table)

print("=== Part 5: Speedup Table ===")
display(speedup_table)

=== Part 5: Runtime Comparison Table ===


,Implementation,N=512,N=1024,N=2048
0,CPU (C),0.297522 sec,3.436814 sec,76.421317 sec
1,Naïve CUDA,0.194 ms,0.939 ms,7.016 ms
2,Optimized CUDA (Tiled),0.144 ms,0.649 ms,4.732 ms


=== Part 5: Speedup Table ===


,N,Speedup (CPU / Naïve CUDA),Speedup (CPU / Optimized CUDA)
0,512,1533.62,2066.13
1,1024,3660.08,5295.55
2,2048,10892.43,16149.90


**Part 6: Using cuBLAS Library**

In [ ]:
%%writefile matrix_cublas.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <cublas_v2.h>

int main(int argc, char **argv) {
    int N = (argc > 1) ? atoi(argv[1]) : 1024;
    size_t size = (size_t)N * (size_t)N * sizeof(float);

    float *hA = (float*)malloc(size);
    float *hB = (float*)malloc(size);
    float *hC = (float*)malloc(size);

    srand(0);
    for (int i = 0; i < N * N; i++) {
        hA[i] = (rand() % 100) / 100.0f;
        hB[i] = (rand() % 100) / 100.0f;
    }

    float *dA, *dB, *dC;
    cudaMalloc((void**)&dA, size);
    cudaMalloc((void**)&dB, size);
    cudaMalloc((void**)&dC, size);

    cudaMemcpy(dA, hA, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dB, hB, size, cudaMemcpyHostToDevice);

    cublasHandle_t handle;
    cublasCreate(&handle);

    float alpha = 1.0f;
    float beta  = 0.0f;

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);

    // Row-major C = A*B  <=>  Column-major C^T = B^T * A^T
    // Using column-major convention with no transpose, swap A and B pointers:
    cublasSgemm(handle,
                CUBLAS_OP_N, CUBLAS_OP_N,
                N, N, N,
                &alpha,
                dB, N,
                dA, N,
                &beta,
                dC, N);

    cudaEventRecord(stop);
    cudaDeviceSynchronize();

    float ms = 0.0f;
    cudaEventElapsedTime(&ms, start, stop);

    cudaMemcpy(hC, dC, size, cudaMemcpyDeviceToHost);

    printf("cuBLAS SGEMM time (N=%d): %.3f ms (%.6f seconds)\n", N, ms, ms / 1000.0);

    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    cublasDestroy(handle);

    cudaFree(dA); cudaFree(dB); cudaFree(dC);
    free(hA); free(hB); free(hC);
    return 0;
}

Writing matrix_cublas.cu


In [ ]:
!nvcc matrix_cublas.cu -o matrix_cublas -O2 -arch=sm_80 -lcublas

In [ ]:
!./matrix_cublas 512
!./matrix_cublas 1024
!./matrix_cublas 2048

cuBLAS SGEMM time (N=512): 120.297 ms (0.120297 seconds)
cuBLAS SGEMM time (N=1024): 68.491 ms (0.068491 seconds)
cuBLAS SGEMM time (N=2048): 69.524 ms (0.069524 seconds)


In [ ]:
import pandas as pd

cpu_sec = {512: 0.297522, 1024: 3.436814, 2048: 76.421317}
naive_ms = {512: 0.194, 1024: 0.939, 2048: 7.016}
tiled_ms = {512: 0.144, 1024: 0.649, 2048: 4.732}
cublas_ms = {512: 120.297, 1024: 68.491, 2048: 69.524}

runtime_table = pd.DataFrame({
    "Implementation": ["CPU (C)", "Naïve CUDA", "Optimized CUDA (Tiled)", "cuBLAS SGEMM"],
    "N=512":  [f"{cpu_sec[512]:.6f} sec",  f"{naive_ms[512]:.3f} ms",  f"{tiled_ms[512]:.3f} ms",  f"{cublas_ms[512]:.3f} ms"],
    "N=1024": [f"{cpu_sec[1024]:.6f} sec", f"{naive_ms[1024]:.3f} ms", f"{tiled_ms[1024]:.3f} ms", f"{cublas_ms[1024]:.3f} ms"],
    "N=2048": [f"{cpu_sec[2048]:.6f} sec", f"{naive_ms[2048]:.3f} ms", f"{tiled_ms[2048]:.3f} ms", f"{cublas_ms[2048]:.3f} ms"],
})

speedup_table = pd.DataFrame({
    "N": [512, 1024, 2048],
    "CPU / Naïve CUDA": [
        cpu_sec[512]  / (naive_ms[512]  / 1000),
        cpu_sec[1024] / (naive_ms[1024] / 1000),
        cpu_sec[2048] / (naive_ms[2048] / 1000),
    ],
    "CPU / Tiled CUDA": [
        cpu_sec[512]  / (tiled_ms[512]  / 1000),
        cpu_sec[1024] / (tiled_ms[1024] / 1000),
        cpu_sec[2048] / (tiled_ms[2048] / 1000),
    ],
    "CPU / cuBLAS": [
        cpu_sec[512]  / (cublas_ms[512]  / 1000),
        cpu_sec[1024] / (cublas_ms[1024] / 1000),
        cpu_sec[2048] / (cublas_ms[2048] / 1000),
    ],
}).round(2)

print("=== Runtime Comparison Table ===")
display(runtime_table)

print("=== Speedup Table (CPU time / GPU time) ===")
display(speedup_table)

=== Runtime Comparison Table ===


,Implementation,N=512,N=1024,N=2048
0,CPU (C),0.297522 sec,3.436814 sec,76.421317 sec
1,Naïve CUDA,0.194 ms,0.939 ms,7.016 ms
2,Optimized CUDA (Tiled),0.144 ms,0.649 ms,4.732 ms
3,cuBLAS SGEMM,120.297 ms,68.491 ms,69.524 ms


=== Speedup Table (CPU time / GPU time) ===


,N,CPU / Naïve CUDA,CPU / Tiled CUDA,CPU / cuBLAS
0,512,1533.62,2066.13,2.47
1,1024,3660.08,5295.55,50.18
2,2048,10892.43,16149.90,1099.21


**Part 8: Creating a Shared Library and Using it in Python**

In [ ]:
%%writefile matrix_lib.cu
#include <cuda_runtime.h>

#define TILE_WIDTH 16

__global__ void matrixMultiplyTiled(const float *A, const float *B, float *C, int N) {
    __shared__ float ds_A[TILE_WIDTH][TILE_WIDTH];
    __shared__ float ds_B[TILE_WIDTH][TILE_WIDTH];

    int tx = threadIdx.x, ty = threadIdx.y;
    int Row = blockIdx.y * TILE_WIDTH + ty;
    int Col = blockIdx.x * TILE_WIDTH + tx;

    float Pvalue = 0.0f;
    int numTiles = (N + TILE_WIDTH - 1) / TILE_WIDTH;

    for (int m = 0; m < numTiles; m++) {
        int A_col = m * TILE_WIDTH + tx;
        int B_row = m * TILE_WIDTH + ty;

        ds_A[ty][tx] = (Row < N && A_col < N) ? A[Row * N + A_col] : 0.0f;
        ds_B[ty][tx] = (B_row < N && Col < N) ? B[B_row * N + Col] : 0.0f;

        __syncthreads();

        for (int k = 0; k < TILE_WIDTH; k++) {
            Pvalue += ds_A[ty][k] * ds_B[k][tx];
        }

        __syncthreads();
    }

    if (Row < N && Col < N) {
        C[Row * N + Col] = Pvalue;
    }
}

extern "C" void gpu_matrix_multiply(float *h_A, float *h_B, float *h_C, int N) {
    size_t size = (size_t)N * (size_t)N * sizeof(float);

    float *d_A, *d_B, *d_C;
    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_B, size);
    cudaMalloc((void**)&d_C, size);

    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    dim3 block(TILE_WIDTH, TILE_WIDTH);
    dim3 grid((N + TILE_WIDTH - 1) / TILE_WIDTH, (N + TILE_WIDTH - 1) / TILE_WIDTH);

    matrixMultiplyTiled<<<grid, block>>>(d_A, d_B, d_C, N);
    cudaDeviceSynchronize();

    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
}

Writing matrix_lib.cu


In [ ]:
!nvcc -Xcompiler -fPIC -shared matrix_lib.cu -o libmatrix.so -O2 -arch=sm_80
!ls -lh libmatrix.so

-rwxr-xr-x 1 root root 984K Jan 31 21:16 libmatrix.so


In [ ]:
import ctypes
import numpy as np
import time

lib = ctypes.cdll.LoadLibrary("./libmatrix.so")

lib.gpu_matrix_multiply.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float32, ndim=1, flags="C_CONTIGUOUS"),
    np.ctypeslib.ndpointer(dtype=np.float32, ndim=1, flags="C_CONTIGUOUS"),
    np.ctypeslib.ndpointer(dtype=np.float32, ndim=1, flags="C_CONTIGUOUS"),
    ctypes.c_int
]
lib.gpu_matrix_multiply.restype = None

N = 1024
A = np.random.rand(N, N).astype(np.float32)
B = np.random.rand(N, N).astype(np.float32)
C = np.zeros((N, N), dtype=np.float32)

start = time.time()
lib.gpu_matrix_multiply(A.ravel(), B.ravel(), C.ravel(), N)
end = time.time()

print(f"Python call to CUDA library completed in {end - start:.4f} seconds")

Python call to CUDA library completed in 0.1299 seconds
